# ALUNOS: 




# CIÊNCIA DE DADOS
## Desafio 2: Previsão da ocupação hospitalar

### Etapas:
- Entendimento do problema 
- Tratamento dos Dados
- Visualização dos dados
- Análise Exploratória dos Dados
- Aprendizado de máquina

O **desafio** consiste na previsão de ocupação hospitalar, focando na análise dos dados de atendimentos do pronto atendimento. 

O **objetivo** é identificar os fatores que podem influenciar ou aumentar o número de entradas de pacientes nesse serviço. As causas para o aumento das entradas podem ser diversas e depender de diferentes variáveis. A partir da análise desses dados, o desafio é desenvolver modelos preditivos que ajudem a antecipar a demanda do pronto atendimento, permitindo uma melhor gestão da ocupação hospitalar e a alocação eficiente de recursos.

Serão fornecidos os seguintes dados para a análise:
- ID do atendimento
- Data/hora da entrada 
- CID
- Data de nascimento
- Sexo 
- Clínica 
- Convenio 
- Classificação de Risco
- Motivo da Alta 
- Data da Alta 
.

## Importar as bibliotecas necessárias e a base de dados. Realizar a visualização da mesma 

## Tratamento dos dados: verificar e corrigir o domínios dos campos, se há dados faltantes, verificar cardinalidade, identificar colunas categóricas, de visualizar as informações estatísticas da base (dados numéricos e não numéricos)

## Visualizando de forma gráfica: utilizar as boas práticas de visualização de dados

## Análise Exploratória dos dados 

**O boxplot**

- Permite visualizar os percentis e valores máximo e mínimo dos dados
- Permite visualizar, além de simetria e dispersão dos dados, valores extremos (outliers)
- O valor máximo e mínimo são calculados baseado no interquartil

**Criando uma matriz para mostrar a correlação de cada par de variáveis**

In [2]:
# Apresentar a matriz de correção da base

In [3]:
# Apresentar a matriz de correlação da base plotando no pairplot

## Visualizando apenas o KDE**
- KDE: Kernel Density Estimation
    - Mede a chance de uma variável aleatória assumir determinado valor
    - A probabilidade é dada pela integral abaixo da curva na faixa de valor selecionada
- Parece uma "versão de linha suavizada" de um histograma

## Heatmap de correlação

## Aprendizado de Máquina

- Definir 3 algorítmos de aprendizado de máquina a ser testado no Desafio


In [4]:
# Importar as bibliotecas necessárias
# Definir as variáveis de input e a target
# Dividir a base de dados em treino e teste


In [5]:
# Treinar o modelo
# Predizer o modelo
# Avaliar o modelo


## Apresente uma conclusão para o desafio, demonstrando graficamente